In [1]:
print("Hello")

Hello


In [2]:
import operator
from typing import Annotated, Any, TypedDict

from langgraph.graph import StateGraph, START, END

In [3]:
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
from langgraph.graph import MessagesState

class InputState(MessagesState):
    string_value: Annotated[str, operator.add]

In [5]:
def append_a(state: InputState)-> InputState:
    state['string_value'] += 'a'
    return state


def append_b(state: InputState)-> InputState:
    state['string_value'] += 'b'
    return state


def append_c(state: InputState)-> InputState:
    state['string_value'] += 'c'
    return state

In [6]:
def combine(state: InputState)-> InputState:
    combined_string = state['string_value']
    print(f"Combined string: {combined_string}")
    final_string = combined_string.upper()
    print(f"Combined string: {final_string}")
    state['string_value'] = final_string
    return state

In [25]:
builder = StateGraph(InputState)

builder.add_node("start_node",lambda state: state)
builder.add_node("append_a", append_a)
builder.add_node("append_b", append_b)
builder.add_node("append_c", append_c)
builder.add_node("combine", combine)

builder.add_edge(START, "start_node")
builder.add_edge("start_node", "append_a")
builder.add_edge("start_node", "append_b")
builder.add_edge("start_node", "append_c")


builder.add_edge("append_a", "combine")
builder.add_edge("append_b", "combine")
builder.add_edge("append_c", "combine")

builder.add_edge("combine", END)

graph = builder.compile()

In [26]:
initial_state = {"string_value": ""}

In [27]:
print(graph.get_graph().draw_mermaid())


%%{init: {'flowchart': {'curve': 'linear'}}}%%
graph TD;
	__start__([<p>__start__</p>]):::first
	start_node(start_node)
	append_a(append_a)
	append_b(append_b)
	append_c(append_c)
	combine(combine)
	__end__([<p>__end__</p>]):::last
	__start__ --> start_node;
	append_a --> combine;
	append_b --> combine;
	append_c --> combine;
	combine --> __end__;
	start_node --> append_a;
	start_node --> append_b;
	start_node --> append_c;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc



In [28]:

result = graph.invoke(initial_state)
print(result)

Combined string: abc
Combined string: ABC
{'messages': [], 'string_value': 'abcABC'}


In [30]:
import operator
import random
from typing import Annotated, Any, TypedDict, List
from langgraph.graph import StateGraph, START, END
from langgraph.graph import MessagesState
from langgraph.types import Send

In [31]:
class MessagesState(MessagesState):
    string_values : Annotated[str, operator.add]
    letters: List[str]

In [32]:
def append_letters(state: MessagesState)-> MessagesState:
    num_nodes = random.randint(5, 10)
    letters = random.choices(["a", "b", "c", "d", "e"], k=num_nodes)
    print(f"Generated letters: {letters}")
    state['letters'] = letters
    return state

def append_letters(state: MessagesState)->MessagesState:
    letter = state['letters']
    state['string_values'] = letter
    return state 


def generate_random_nodes(state: MessagesState)-> MessagesState:
    letters = state['letters']
    return [Send("append_letters", {"letter": letter}) for letter in letters]

In [33]:
def combine_strings(state: MessagesState) -> Any:
    combined_string = state["string_value"]
    print(f"combine_strings: Combined string_value='{combined_string}'")
    final_string = combined_string.upper()
    print(f"combine_strings: Final string_value='{final_string}'")
    return {"string_value": final_string}